In [1]:
import numpy as np
from scipy.io import loadmat, savemat
import matplotlib.pyplot as plt
from PIL import Image
import os
import scipy

In [2]:
def load_keypoints(filepath):
    """Load keypoints and descriptors from .mat file"""
    data = loadmat(filepath)
    return data['kp'], data['desc']

def find_matches(desc1, desc2, ratio_threshold=0.75):
    """Find matches between two sets of descriptors using ratio test"""
    # Compute distances between all descriptors
    distances = np.linalg.norm(desc1[:, np.newaxis] - desc2, axis=2)
    
    # For each descriptor in desc1, find the two closest matches in desc2
    best_matches = np.argmin(distances, axis=1)
    best_distances = np.min(distances, axis=1)
    
    # Find second best matches
    distances[np.arange(len(desc1)), best_matches] = np.inf
    second_best_distances = np.min(distances, axis=1)
    
    # Apply ratio test
    good_matches_mask = best_distances < (ratio_threshold * second_best_distances)
    matches = np.column_stack((np.where(good_matches_mask)[0], 
                             best_matches[good_matches_mask]))
    return matches

def estimate_homography(points1, points2):
    """Estimate homography matrix using DLT algorithm"""
    if len(points1) < 4:
        return None
    
    # Normalize points
    mean1 = np.mean(points1, axis=0)
    mean2 = np.mean(points2, axis=0)
    scale1 = np.sqrt(2) / np.mean(np.linalg.norm(points1 - mean1, axis=1))
    scale2 = np.sqrt(2) / np.mean(np.linalg.norm(points2 - mean2, axis=1))
    
    T1 = np.array([[scale1, 0, -scale1*mean1[0]],
                   [0, scale1, -scale1*mean1[1]],
                   [0, 0, 1]])
    T2 = np.array([[scale2, 0, -scale2*mean2[0]],
                   [0, scale2, -scale2*mean2[1]],
                   [0, 0, 1]])
    
    # Build matrix for DLT
    A = []
    norm_points1 = np.hstack((points1, np.ones((len(points1), 1))))
    norm_points2 = np.hstack((points2, np.ones((len(points2), 1))))
    norm_points1 = (T1 @ norm_points1.T).T
    norm_points2 = (T2 @ norm_points2.T).T
    
    for i in range(len(points1)):
        x, y = norm_points1[i, 0], norm_points1[i, 1]
        u, v = norm_points2[i, 0], norm_points2[i, 1]
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
        A.append([x, y, 1, 0, 0, 0, -u*x, -u*y, -u])
    
    A = np.array(A)
    
    # Solve using SVD
    _, _, Vt = np.linalg.svd(A)
    H = Vt[-1].reshape(3, 3)
    
    # Denormalize
    H = np.linalg.inv(T2) @ H @ T1
    return H / H[2, 2]

def transform_points(points, H):
    """Transform points using homography matrix"""
    # Convert to homogeneous coordinates
    points_h = np.hstack((points, np.ones((len(points), 1))))
    # Transform points
    transformed_h = (H @ points_h.T).T
    # Convert back to euclidean coordinates
    transformed = transformed_h[:, :2] / transformed_h[:, 2:]
    return transformed

In [ ]:
import numpy as np
import scipy.io
from scipy.spatial.distance import cdist
from scipy.linalg import svd

def load_keypoints(file_path):
    """Wczytaj kluczowe punkty i deskryptory z pliku .mat"""
    data = scipy.io.loadmat(file_path)
    kp = data['kp']  # Nx2 macierz współrzędnych kluczowych punktów
    desc = data['desc']  # NxD macierz deskryptorów
    return kp, desc

def match_keypoints(desc1, desc2, threshold=0.75):
    """Dopasowanie kluczowych punktów na podstawie odległości między deskryptorami"""
    distances = cdist(desc1, desc2, metric='euclidean')
    matches = []
    for i in range(distances.shape[0]):
        sorted_indices = np.argsort(distances[i])
        if distances[i, sorted_indices[0]] < threshold * distances[i, sorted_indices[1]]:
            matches.append((i, sorted_indices[0]))
    return np.array(matches)

def compute_homography(src_pts, dst_pts):
    """Oblicz macierz homografii z punktów dopasowanych"""
    assert src_pts.shape == dst_pts.shape
    num_points = src_pts.shape[0]
    if num_points < 4:
        raise ValueError("Do obliczenia homografii wymagane są co najmniej 4 punkty.")

    A = []
    for i in range(num_points):
        x, y = src_pts[i]
        x_p, y_p = dst_pts[i]
        A.append([-x, -y, -1, 0, 0, 0, x*x_p, y*x_p, x_p])
        A.append([0, 0, 0, -x, -y, -1, x*y_p, y*y_p, y_p])
    A = np.array(A)

    # Rozkład SVD
    U, S, Vt = svd(A)
    H = Vt[-1].reshape((3, 3))
    return H / H[2, 2]  # Normalizacja

In [5]:
import math

def calculate_ransac_iterations(w, n, p=0.99):
    """
    Oblicz wymaganą liczbę iteracji dla RANSAC.
    
    Args:
        w (float): Szacowany udział inlinerów (np. 0.5 dla 50%).
        n (int): Minimalna liczba punktów do zbudowania modelu (np. 4 dla homografii).
        p (float): Prawdopodobieństwo sukcesu (domyślnie 0.99).
    
    Returns:
        int: Liczba iteracji.
    """
    if w <= 0 or w > 1:
        raise ValueError("w (udział inlinerów) musi być w zakresie (0, 1].")
    if n <= 0:
        raise ValueError("n (liczba punktów) musi być większe od 0.")
    
    return math.ceil(math.log(1 - p) / math.log(1 - w**n))

# Przykład użycia
w = 0.3  # Szacowany udział inlinerów (50%)
n = 4    # Minimalna liczba punktów dla homografii
p = 0.99 # Prawdopodobieństwo sukcesu (99%)

k = calculate_ransac_iterations(w, n, p)
print(f"Liczba iteracji RANSAC: {k}")

Liczba iteracji RANSAC: 567
